# How to use functions with a knowledge base

This notebook builds on the concepts in the [argument generation]('How_to_generate_function_arguments_with_chat_models.ipynb') notebook, by creating an agent with access to a knowledge base and two functions that it can call based on the user requirement.

We'll create an agent that uses data from arXiv to answer questions about academic subjects. It has two functions at its disposal:
- **get_articles**: A function that gets arXiv articles on a subject and summarizes them for the user with links.
- **read_article_and_summarize**: This function takes one of the previously searched articles, reads it in its entirety and summarizes the core argument, evidence and conclusions.

This will get you comfortable with a multi-function workflow that can choose from multiple services, and where some of the data from the first function is persisted to be used by the second.

## Walkthrough

This cookbook takes you through the following workflow:

- **Search utilities:** Creating the two functions that access arXiv for answers.
- **Configure Agent:** Building up the Agent behaviour that will assess the need for a function and, if one is required, call that function and present results back to the agent.
- **arXiv conversation:** Put all of this together in live conversation.


In [1]:
!pip install scipy
!pip install tenacity
!pip install tiktoken==0.3.3
!pip install termcolor 
!pip install openai
!pip install requests
!pip install arxiv
!pip install pandas
!pip install PyPDF2
!pip install tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, http://mirrors.aliyun.com/pypi/simple/, http://pypi.douban.com/simple, http://pypi.mirrors.ustc.edu.cn/simple/
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, http://mirrors.aliyun.com/pypi/simple/, http://pypi.douban.com/simple, http://pypi.mirrors.ustc.edu.cn/simple/
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, http://mirrors.aliyun.com/pypi/simple/, http://pypi.douban.com/simple, http://pypi.mirrors.ustc.edu.cn/simple/
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, http://mirrors.aliyun.com/pypi/simple/, http://pypi.douban.com/simple, http://pypi.mirrors.ustc.edu.cn/simple/
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, http://mirrors.aliyun.com/pypi/simple/, http://pypi.douban.com/simple, http://pypi.mirrors.ustc.edu.cn/simple/
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, http://mirrors.aliyun.com/pypi/simple/, http://pypi.douban.com/simp

In [263]:
import arxiv
import ast
import concurrent
from csv import writer
from IPython.display import display, Markdown, Latex
import json
import openai
import os
import pandas as pd
from PyPDF2 import PdfReader
import requests
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
from tqdm import tqdm
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

openai.api_base = "https://dev.chatwithoracle.com/api/v1"

bare_base = openai.api_base.replace("/v1", "")
print(bare_base)

https://dev.chatwithoracle.com/api


## Search utilities

We'll first set up some utilities that will underpin our two functions.

Downloaded papers will be stored in a directory (we use ```./data/papers``` here). We create a file ```arxiv_library.csv``` to store the embeddings and details for downloaded papers to retrieve against using ```summarize_text```.

In [227]:
# Set a directory to store downloaded papers
doc_root = 'data'

data_dir = os.path.join(doc_root, 'papers')
paper_dir_filepath = "./data/arxiv_library.csv"

# Generate a blank dataframe where we can store downloaded files
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)


In [238]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = openai.Embedding.create(input=text, model=EMBEDDING_MODEL)
    return response

def build_paper_head(paper_head, prefix = None) :
    # f"Title: {strings[0]['title']}\nAuthor: {strings[0]['authors']}\nPublished: {strings[0]['published']}\n" 
    head = ''
    if prefix is not None: 
        head = f"#{prefix}. "
    
    head += f"**[{paper_head['title']}]({paper_head['article_url']})**\n- **Author**"

    authors = paper_head['authors']
    if isinstance(authors, str) :
        authors = eval(authors)
    
    for author in authors:
        head += f"\n  - {author.name}"
    head += f"\n- **Published**: {paper_head['published']}"
    return head

def search_article_list(query, library=paper_dir_filepath, top_k=5, recent_days=None):
    """
    This function gets the top_k articles based on a user's query, sorted by relevance and recency. 
    Return the title, summary, authors and published date for each article
    """
    search = arxiv.Search(
        query=query, max_results=top_k, sort_by=arxiv.SortCriterion.Relevance
    )

    result_list = []
    for result in search.results():
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})
        result_dict.update({"authors": result.authors})
        result_dict.update({"published": result.published})

        # Taking the first url provided
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)
    
        # Store references in library file
        response = embedding_request(text=result.title)
        file_reference = [
            result.title,
            result.authors,
            result.published,
            result_dict['article_url'],
            result.download_pdf(data_dir),
            response["data"][0]["embedding"],
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
            
    # compose result manually
    if len(result_list) == 0:
        return "No results found"
    result_buffer = f"Search results for '**{query}**': \n\n"
    for idx, result in enumerate(result_list):
        result_buffer += build_paper_head(result, idx) + "\n\n"

    return result_buffer

def get_articles(query, library=paper_dir_filepath, top_k=1):
    """This function gets the top_k articles based on a user's query, sorted by relevance.
    It also downloads the files and stores them in arxiv_library.csv to be retrieved by the read_article_and_summarize.
    """
    search = arxiv.Search(
        query=f"{query}:title", max_results=top_k, sort_by=arxiv.SortCriterion.Relevance
    )
    result_list = []
    
    nresult = 0
    for result in search.results():
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})
        result_dict.update({"authors": result.authors})
        result_dict.update({"published": result.published})

        # Taking the first url provided
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)

        # Store references in library file
        response = embedding_request(text=result.title)
        file_reference = [
            result.title,
            result.authors,
            result.published,
            result_dict['article_url'],
            result.download_pdf(data_dir),
            response["data"][0]["embedding"],
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
        nresult += 1

    print(f"Searching for '{query}' with topk={top_k}, get {nresult} results\n")
    print(f"Top-ranked paper's title: {result_list[0]['title']}")
    return result_list


In [256]:
# Test that the search is working
# result_output = get_articles("ppo reinforcement learning")
result_output = get_articles("Prompt Engineering")


Searching for 'Prompt Engineering' with topk=1, get 1 results

Top-ranked paper's title: Prompt Agnostic Essay Scorer: A Domain Generalization Approach to Cross-prompt Automated Essay Scoring


In [35]:
# details about the author
for author in result_output[0]['authors'] :
    # print(f"name:{author.name}, affiliation:{author.arxiv.affiliation}, email:{author.email}")
    print(f"name:{author.name}")
    print("------------------")

name:Yi-Lin Tuan
------------------
name:Jinzhi Zhang
------------------
name:Yujia Li
------------------
name:Hung-yi Lee
------------------


In [68]:
a = ([3,4], 0.4)
print(a)

authors = "[arxiv.Result.Author('Yi-Lin Tuan'), arxiv.Result.Author('Jinzhi Zhang'), arxiv.Result.Author('Yujia Li'), arxiv.Result.Author('Hung-yi Lee')]"

for author in eval(authors):
    print(author.name)

([3, 4], 0.4)
Yi-Lin Tuan
Jinzhi Zhang
Yujia Li
Hung-yi Lee


In [230]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
    rel_th : float = 0.95
) -> list[str]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = embedding_request(query)
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        ({"filepath" : row["filepath"], "title": row["title"], 
          "authors" : row["authors"], "published" : row["published"], 
          "article_url" : row["article_url"]}, 
         relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    if relatednesses[top_n-1] > rel_th :
        print(f"Find paper with relatedness {relatednesses[top_n-1]} > {rel_th}!\n")
        return strings[:top_n]
    return None


In [242]:
def read_pdf(filepath):
    """Takes a filepath to a PDF and returns a string of the PDF's contents"""
    # creating a pdf reader object
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text

# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


def extract_chunk(content, template_prompt):
    """This function applies a prompt to some input content. In this case it returns a summarize chunk of text"""
    prompt = template_prompt + content
    response = openai.ChatCompletion.create(
        model=GPT_MODEL, messages=[{"role": "user", "content": prompt}], temperature=0
    )
    return response["choices"][0]["message"]["content"]

def summarize_text(query):
    """This function does the following:
    - Reads in the arxiv_library.csv file in including the embeddings
    - Finds the closest file to the user's query
    - Scrapes the text out of the file and chunks it
    - Summarizes each chunk in parallel
    - Does one final summary and returns this to the user"""

    # A prompt to dictate how the recursive summarizations should approach the input paper
    summary_prompt = """Summarize this text from an academic paper. Extract any key points with reasoning.\n\nContent:"""

    # If the library is empty (no searches have been performed yet), we perform one and download the results
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("No papers searched yet, downloading first.")
        result_list = get_articles(query)
        print("Papers downloaded, continuing")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()

    # rank by relatedness
    library_df.columns = ["title", "authors", "published", "article_url", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1, rel_th=0.95)

    if strings is None : # no-related papers in local repository, search arXiv again
        result_list = get_articles(query)
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
        library_df.columns = ["title", "authors", "published", "article_url", "filepath", "embedding"]
        library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
        strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    
    assert strings is not None, "No papers found"
    
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0]['filepath'])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # Chunk up the document into 1500 token chunks
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # Parallel process the summaries
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=len(text_chunks)
    ) as executor:
        futures = [
            executor.submit(extract_chunk, chunk, summary_prompt)
            for chunk in text_chunks
        ]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # Final summary
    print("Summarizing into overall summary")
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary collated from this collection of key points extracted from an academic paper.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions.
                        Key points:\n{results}\nSummary:\n""",
            }
        ],
        temperature=0,
    )
    
    # attach title, author & published
    paper_head = build_paper_head(strings[0])
    response["choices"][0]["message"]["content"] = paper_head + "\n\n" + response["choices"][0]["message"]["content"]
    return response


In [81]:
# Test the summarize_text function works
chat_test_response = summarize_text("PPO reinforcement learning sequence generation")


Chunking text from paper
Summarizing each chunk of text


100%|██████████| 4/4 [00:08<00:00,  2.14s/it]


Summarizing into overall summary
{'filepath': 'data/papers/1808.07982v1.Proximal_Policy_Optimization_and_its_Dynamic_Version_for_Sequence_Generation.pdf', 'title': 'Proximal Policy Optimization and its Dynamic Version for Sequence Generation', 'authors': "[arxiv.Result.Author('Yi-Lin Tuan'), arxiv.Result.Author('Jinzhi Zhang'), arxiv.Result.Author('Yujia Li'), arxiv.Result.Author('Hung-yi Lee')]", 'published': '2018-08-24 02:14:43+00:00'}


In [82]:
print(chat_test_response["choices"][0]["message"]["content"])


- **Title**: Proximal Policy Optimization and its Dynamic Version for Sequence Generation
- **Author**
  - Yi-Lin Tuan
  - Jinzhi Zhang
  - Yujia Li
  - Hung-yi Lee
- **Published**: 2018-08-24 02:14:43+00:00

Core Argument:
- The paper discusses the use of Proximal Policy Optimization (PPO) in sequence generation tasks, specifically in the context of chit-chat chatbots.
- The authors argue that PPO is a more efficient reinforcement learning algorithm compared to policy gradient, commonly used in text generation tasks.
- They propose a dynamic approach for PPO (PPO-dynamic) and demonstrate its efficacy in synthetic experiments and chit-chat chatbot tasks.

Evidence:
- PPO-dynamic achieves high precision scores comparable to other algorithms in a synthetic counting task.
- PPO-dynamic shows faster progress and a more stable learning curve compared to PPO in both synthetic and chit-chat chatbot tasks.
- In the chit-chat chatbot task, PPO-dynamic achieves a slightly higher BLEU-2 score tha

## Configure Agent

We'll create our agent in this step, including a ```Conversation``` class to support multiple turns with the API, and some Python functions to enable interaction between the ```ChatCompletion``` API and our knowledge base functions.

In [261]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        # base = openai.api_base or "https://api.openai.com"
        base = "https://api.openai.com"
        base = base.replace("v1/", "")
        response = requests.post(
            f"{base}/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [233]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [234]:
# Initiate our get_articles and read_article_and_summarize functions
arxiv_functions = [
    {
        "name": "get_articles",
        "description": """Use this function to download an academic paper from arXiv to answer user questions.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            User query in JSON. Responses should be summarized and should include the article URL reference
                            """,
                }
            },
            "required": ["query"],
        }
    },

    {
        "name": "search_and_return_articles", 
        "description": """Use this function to search arXiv database for a list of articles related to user query and return, 
        argument 'top_k' should be an integer for the number of articles to return,
        argument 'recency_days' should be an integer for the number of days since the article was published""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""Subject of the article in plain text"""
                },
                "top_k" : {
                    "type": "integer",
                    "description": f"""Number of articles to return"""
                }, 
                "recent_days" : {
                    "type": "integer",
                    "description": f"""Number of days since the article was published"""
                }
            }, 
            "required": ["query"],
        },
    },
    
    {
        "name": "read_article_and_summarize",
        "description": """Use this function to read whole papers and provide a summary for users.
        You should NEVER call this function before search_and_return_articles has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            Description of the article in plain text based on the user's query
                            """,
                }
            },
            "required": ["query"],
        },
    }
]


In [235]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function {full_message}")
        return call_arxiv_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()


def call_arxiv_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""
    func_name = full_message["message"]["function_call"]["name"]
    print(f'Get function name: {func_name}')
    if func_name == "get_articles":
        try:
            parsed_output = json.loads(
                full_message["message"]["function_call"]["arguments"]
            )
            print("Getting search results")
            results = get_articles(parsed_output["query"])
        except Exception as e:
            print(parsed_output)
            print(f"Function execution failed")
            print(f"Error message: {e}")
        messages.append(
            {
                "role": "function",
                "name": full_message["message"]["function_call"]["name"],
                "content": str(results),
            }
        )
        try:
            print("Got search results, summarizing content")
            response = chat_completion_request(messages)
            return response.json()
        except Exception as e:
            print(type(e))
            raise Exception("Function chat request failed")

    elif func_name == "read_article_and_summarize":
        parsed_output = json.loads(
            full_message["message"]["function_call"]["arguments"]
        )
        print("Finding and reading paper")
        summary = summarize_text(parsed_output["query"])
        return summary
    elif func_name == "search_and_return_articles" :
        arguments = eval(full_message["message"]["function_call"]["arguments"])
        article_lst = search_article_list(arguments['query'], top_k=arguments['top_k'])
        return article_lst
    else:
        raise Exception("Function does not exist and cannot be called")


## arXiv conversation

Let's put this all together by testing our functions out in conversation.

In [249]:
# Start with a system message
# paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
# You summarize the papers clearly so the customer can decide which to read to answer their question.
# You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
# Begin!"""

paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions. 
You search for a list of papers to answer user questions. You read and summarize the paper clearly according to user provided topics.
Please summarize in clear and concise format. Begin!"""

paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [250]:
# Add a user message
paper_conversation.add_message("user", "Hi, what is the most recent 5 papers on prompt enginnering?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)

if isinstance(chat_response, str):
    assistant_message = chat_response
else :
    assistant_message = chat_response["choices"][0]["message"]["content"]
    
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Function generation requested, calling function {'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'search_and_return_articles', 'arguments': '{\n  "query": "prompt engineering",\n  "top_k": 5,\n  "recent_days": 365\n}'}}, 'finish_reason': 'function_call'}
Get function name: search_and_return_articles


Search results for '**prompt engineering**': 

#0. **[Prompting AI Art: An Investigation into the Creative Skill of Prompt Engineering](http://arxiv.org/abs/2303.13534v1)**
- **Author**
  - Jonas Oppenlaender
  - Rhema Linder
  - Johanna Silvennoinen
- **Published**: 2023-03-13 14:25:56+00:00

#1. **[Prompt Space Optimizing Few-shot Reasoning Success with Large Language Models](http://arxiv.org/abs/2306.03799v1)**
- **Author**
  - Fobo Shi
  - Peijun Qing
  - Dong Yang
  - Nan Wang
  - Youbo Lei
  - Haonan Lu
  - Xiaodong Lin
- **Published**: 2023-06-06 15:43:16+00:00

#2. **[Jailbreaking ChatGPT via Prompt Engineering: An Empirical Study](http://arxiv.org/abs/2305.13860v1)**
- **Author**
  - Yi Liu
  - Gelei Deng
  - Zhengzi Xu
  - Yuekang Li
  - Yaowen Zheng
  - Ying Zhang
  - Lida Zhao
  - Tianwei Zhang
  - Yang Liu
- **Published**: 2023-05-23 09:33:38+00:00

#3. **[A Prompt Pattern Catalog to Enhance Prompt Engineering with ChatGPT](http://arxiv.org/abs/2302.11382v1)**
- **Author**
  - Jules White
  - Quchen Fu
  - Sam Hays
  - Michael Sandborn
  - Carlos Olea
  - Henry Gilbert
  - Ashraf Elnashar
  - Jesse Spencer-Smith
  - Douglas C. Schmidt
- **Published**: 2023-02-21 12:42:44+00:00

#4. **[A Simple Zero-shot Prompt Weighting Technique to Improve Prompt Ensembling in Text-Image Models](http://arxiv.org/abs/2302.06235v1)**
- **Author**
  - James Urquhart Allingham
  - Jie Ren
  - Michael W Dusenberry
  - Jeremiah Zhe Liu
  - Xiuye Gu
  - Yin Cui
  - Dustin Tran
  - Balaji Lakshminarayanan
- **Published**: 2023-02-13 10:19:58+00:00



In [243]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Can you read the paper of Prompt AI art, and give me a summary?",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)

if isinstance(updated_response, str):
    assistant_message = updated_response
else :
    assistant_message = updated_response["choices"][0]["message"]["content"]

display(Markdown(assistant_message))


Function generation requested, calling function {'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'read_article_and_summarize', 'arguments': '{\n  "query": "Prompting AI Art: An Investigation into the Creative Skill of Prompt Engineering"\n}'}}, 'finish_reason': 'function_call'}
Get function name: read_article_and_summarize
Finding and reading paper
Find paper with relatedness 1 > 0.95!

Chunking text from paper
Summarizing each chunk of text


100%|██████████| 11/11 [00:15<00:00,  1.43s/it]


Summarizing into overall summary


**[Prompting AI Art: An Investigation into the Creative Skill of Prompt Engineering](http://arxiv.org/abs/2303.13534v1)**
- **Author**
  - Jonas Oppenlaender
  - Rhema Linder
  - Johanna Silvennoinen
- **Published**: 2023-03-13 14:25:56+00:00

Core Argument:
- This academic paper investigates the creative skill of prompt engineering in the context of AI art.
- Prompt engineering involves iteratively crafting prompts to generate and improve digital images using AI.
- The paper presents three studies conducted with participants recruited from a crowdsourcing platform to explore their ability to recognize the quality of prompts, write prompts, and improve their prompts.
- Prompt engineering is a learned skill that requires expertise and practice.

Evidence:
- Participants were able to assess the quality of prompts and images, with this ability increasing with their experience and interest in art.
- Participants were able to write prompts in rich descriptive language, but lacked the specific vocabulary needed to apply a certain style to the generated images.
- Participants' familiarity with art was negatively correlated with the average error in consistency, suggesting that participants with more art experience were better at predicting the quality of the generated artworks.
- Participants were able to generate effective input prompts for text-to-image systems, demonstrating an understanding of how to maximize the visual attractiveness and aesthetic qualities of the generated artworks.
- Participants made changes to their prompts by adding more tokens, particularly nouns and adjectives.
- Over half of the revised sets of images showed no improvement in image quality, while a third were better and 15% were worse.
- Participants were able to make improvements by adding more details and improving colors and contrast.
- Changes in the style of the images were rare, with most revised sets being in the same or similar style.

Conclusions:
- Prompt engineering is a skill that requires expertise and knowledge of certain keywords and phrases.
- Prompt engineering could become an expert skill limited to a small group of professionals or a common practice used in everyday life.
- Prompt engineering may become obsolete as generative systems improve their ability to understand user intent.
- Prompt engineering could become a personal signature or curation skill in the future.
- Recommendations for conducting text-to-image experiments with crowd workers include providing clear instructions, explaining keywords and key phrases, and considering the limitations of crowd workers.
- The findings offer a deeper understanding of prompt engineering and open up avenues for future research in this field.

In [244]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "OK, give me a summary of the paper about Jailbreaking ChatGPT.",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)

if isinstance(updated_response, str):
    assistant_message = updated_response
else :
    assistant_message = updated_response["choices"][0]["message"]["content"]

display(Markdown(assistant_message))

Function generation requested, calling function {'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'read_article_and_summarize', 'arguments': '{\n  "query": "Jailbreaking ChatGPT via Prompt Engineering: An Empirical Study"\n}'}}, 'finish_reason': 'function_call'}
Get function name: read_article_and_summarize
Finding and reading paper
Find paper with relatedness 1 > 0.95!

Chunking text from paper
Summarizing each chunk of text


100%|██████████| 9/9 [00:15<00:00,  1.71s/it]


Summarizing into overall summary


**[Jailbreaking ChatGPT via Prompt Engineering: An Empirical Study](http://arxiv.org/abs/2305.13860v1)**
- **Author**
  - Yi Liu
  - Gelei Deng
  - Zhengzi Xu
  - Yuekang Li
  - Yaowen Zheng
  - Ying Zhang
  - Lida Zhao
  - Tianwei Zhang
  - Yang Liu
- **Published**: 2023-05-23 09:33:38+00:00

Core Argument:
- The academic paper investigates the concept of "jailbreaking" Large Language Models (LLMs), specifically focusing on the C HATGPT model.
- The study aims to answer three research questions: the number of different prompt types that can jailbreak LLMs, the effectiveness of jailbreak prompts in bypassing LLM constraints, and the resilience of C HATGPT against these jailbreak prompts.

Evidence:
- The paper discusses the popularity and adoption of LLMs like C HATGPT and the limitations imposed by OpenAI to address concerns of misuse.
- The study collects 78 verified jailbreak prompts and develops a classification model to analyze their distribution.
- The jailbreak prompts are tested on C HATGPT versions 3.5 and 4.0 using a dataset of 3,120 jailbreak questions across eight prohibited scenarios.
- The paper evaluates the resistance of C HATGPT against jailbreak prompts and identifies factors that affect their capabilities.
- The paper compares the protection power of GPT-3.5-TURBO and GPT-4 and evaluates the strength of jailbreak prompts.

Conclusions:
- The study provides insights into the types and capabilities of jailbreak prompts, the effectiveness of these prompts in bypassing LLM restrictions, and the resilience of C HATGPT against jailbreak attempts.
- The paper highlights the importance of prompt structures in jailbreaking LLMs and discusses the challenges of generating robust jailbreak prompts and preventing prompt-based jailbreaks.
- The paper emphasizes the need to align content policy strength with real-world laws and ethical standards.
- The study suggests possible future research directions to enhance the robustness and security of language models like ChatGPT.

In [245]:
# Add a user message
paper_conversation.add_message("user", "Give me recent 5 papers about parameter efficient tuning?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)

if isinstance(chat_response, str):
    assistant_message = chat_response
else :
    assistant_message = chat_response["choices"][0]["message"]["content"]
    
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Function generation requested, calling function {'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'search_and_return_articles', 'arguments': '{\n  "query": "parameter efficient tuning",\n  "top_k": 5\n}'}}, 'finish_reason': 'function_call'}
Get function name: search_and_return_articles


Search results for '**parameter efficient tuning**': 

#0. **[Parameter-Efficient Fine-Tuning Design Spaces](http://arxiv.org/abs/2301.01821v1)**
- **Author**
  - Jiaao Chen
  - Aston Zhang
  - Xingjian Shi
  - Mu Li
  - Alex Smola
  - Diyi Yang
- **Published**: 2023-01-04 21:00:18+00:00

#1. **[Parameter-Efficient Tuning on Layer Normalization for Pre-trained Language Models](http://arxiv.org/abs/2211.08682v3)**
- **Author**
  - Wang Qi
  - Yu-Ping Ruan
  - Yuan Zuo
  - Taihao Li
- **Published**: 2022-11-16 05:31:49+00:00

#2. **[Residual Prompt Tuning: Improving Prompt Tuning with Residual Reparameterization](http://arxiv.org/abs/2305.03937v1)**
- **Author**
  - Anastasia Razdaibiedina
  - Yuning Mao
  - Rui Hou
  - Madian Khabsa
  - Mike Lewis
  - Jimmy Ba
  - Amjad Almahairi
- **Published**: 2023-05-06 05:35:14+00:00

#3. **[Scaling & Shifting Your Features: A New Baseline for Efficient Model Tuning](http://arxiv.org/abs/2210.08823v3)**
- **Author**
  - Dongze Lian
  - Daquan Zhou
  - Jiashi Feng
  - Xinchao Wang
- **Published**: 2022-10-17 08:14:49+00:00

#4. **[Scattered or Connected? An Optimized Parameter-efficient Tuning Approach for Information Retrieval](http://dx.doi.org/10.1145/3511808.3557445)**
- **Author**
  - Xinyu Ma
  - Jiafeng Guo
  - Ruqing Zhang
  - Yixing Fan
  - Xueqi Cheng
- **Published**: 2022-08-21 08:56:02+00:00



In [246]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Thanks, please summary the paper about Residual Prompt Tuning.",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)

if isinstance(updated_response, str):
    assistant_message = updated_response
else :
    assistant_message = updated_response["choices"][0]["message"]["content"]

display(Markdown(assistant_message))

Function generation requested, calling function {'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'read_article_and_summarize', 'arguments': '{\n  "query": "Residual Prompt Tuning: Improving Prompt Tuning with Residual Reparameterization"\n}'}}, 'finish_reason': 'function_call'}
Get function name: read_article_and_summarize
Finding and reading paper
Find paper with relatedness 1 > 0.95!

Chunking text from paper
Summarizing each chunk of text


100%|██████████| 8/8 [00:12<00:00,  1.51s/it]


Summarizing into overall summary


**[Residual Prompt Tuning: Improving Prompt Tuning with Residual Reparameterization](http://arxiv.org/abs/2305.03937v1)**
- **Author**
  - Anastasia Razdaibiedina
  - Yuning Mao
  - Rui Hou
  - Madian Khabsa
  - Mike Lewis
  - Jimmy Ba
  - Amjad Almahairi
- **Published**: 2023-05-06 05:35:14+00:00

Core Argument:
- The academic paper introduces a method called Residual Prompt Tuning that improves the performance and stability of prompt tuning for parameter-efficient tuning of pretrained language models.

Evidence:
- Residual Prompt Tuning reparameterizes soft prompt embeddings using a shallow network with a residual connection, which significantly outperforms prompt tuning on the SuperGLUE benchmark.
- The method allows for a reduction in prompt length without sacrificing performance and is robust to different learning rates and prompt initializations.
- The experiments are conducted on NLU tasks from the SuperGLUE benchmark using BERT and T5 models, comparing Residual Prompt Tuning against other prompt reparameterization techniques and parameter-efficient tuning methods.
- Residual Prompt Tuning outperforms other reparameterization techniques and achieves competitive performance compared to fine-tuning.
- The use of residual connections and parameter sharing in the reparameterization network improves performance and offers parameter efficiency and knowledge sharing benefits.

Conclusions:
- Residual Prompt Tuning is an effective method for improving the performance of transfer learning-based models in natural language processing tasks.
- Residual Prompt Tuning is robust to hyperparameter choice, speeds up convergence, and is effective in few-shot settings.
- The method requires the same number of additional parameters during inference as the original prompt tuning method, but it requires 25 times fewer extra parameters compared to adapter-based methods.
- The paper acknowledges the contributions of other researchers in the field and discusses the ethical implications of the proposed method.

In [262]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Thanks, please summary the paper about Fine-Tuning Design Spaces.",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)

if isinstance(updated_response, str):
    assistant_message = updated_response
else :
    assistant_message = updated_response["choices"][0]["message"]["content"]

display(Markdown(assistant_message))

Function generation requested, calling function {'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'read_article_and_summarize', 'arguments': '{\n  "query": "Fine-Tuning Design Spaces"\n}'}}, 'finish_reason': 'function_call'}
Get function name: read_article_and_summarize
Finding and reading paper
Find paper with relatedness 0.96118840857324 > 0.95!

Chunking text from paper
Summarizing each chunk of text


100%|██████████| 11/11 [00:15<00:00,  1.39s/it]


Summarizing into overall summary


**[Parameter-Efficient Fine-Tuning Design Spaces](http://arxiv.org/abs/2301.01821v1)**
- **Author**
  - Jiaao Chen
  - Aston Zhang
  - Xingjian Shi
  - Mu Li
  - Alex Smola
  - Diyi Yang
- **Published**: 2023-01-04 21:00:18+00:00

The academic paper discusses parameter-efficient fine-tuning in natural language processing tasks. The authors propose a parameter-efficient fine-tuning design paradigm and discover design patterns that can be applied to different experimental settings. They introduce parameter-efficient fine-tuning design spaces that parameterize tuning structures and strategies. The design spaces are characterized by layer grouping, trainable parameter allocation, tunable groups, and strategy assignment. The authors progressively refine the design space based on the model quality of each design choice and make greedy selections. They discover design patterns such as grouping layers in a spindle pattern, allocating trainable parameters uniformly, tuning all groups, and assigning proper tuning strategies to different groups. These design patterns result in new parameter-efficient fine-tuning methods that consistently outperform existing strategies. The authors also show that these methods are applicable to different backbone models and tasks in natural language processing. The contributions of the paper include introducing parameter-efficient fine-tuning design spaces, discovering design patterns through experiments, and developing parameter-efficient fine-tuning methods that outperform existing strategies.